<a href="https://colab.research.google.com/github/aframuneer26/Deep_Learning_Experiments/blob/main/DL_EXP_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split, Subset
from PIL import Image
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

# Data
transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
dataset = Subset(datasets.CIFAR10('./data',True,download=True,transform=transform), range(500))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Training
def train_and_evaluate(model, name, num_epochs=5):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(num_epochs):
        model.train()
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)

            # If using GoogLeNet, ensure only the main output is used for loss
            if name == "googlenet":
                outputs = outputs[0]  # Only the main output, not the auxiliary outputs

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)

                if name == "googlenet":
                    outputs = outputs[0]

                _, predicted = outputs.max(1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)

        print(f"{name} Epoch {epoch+1}/{num_epochs} - Train Acc: {100*correct/total:.2f}%, Val Acc: {100*correct_val/total_val:.2f}%")

    return model

# Models
def get_model(name):
    if name == "vgg":
        m = models.vgg16(pretrained=True)
        m.classifier[6] = nn.Linear(4096, 10)
    elif name == "resnet":
        m = models.resnet18(pretrained=True)
        m.fc = nn.Linear(m.fc.in_features, 10)
    elif name == "googlenet":
        m = models.googlenet(pretrained=True, aux_logits=True)
        m.fc = nn.Linear(m.fc.in_features, 10)
    else:
        raise ValueError("Unknown model")
    return m

results = {}
trained_models = {}
for name in ["vgg", "resnet", "googlenet"]:
    print(f"\nTraining {name.upper()} on CIFAR-10...")
    model = get_model(name)
    trained_models[name] = train_and_evaluate(model, name)

# Plot
plt.figure(figsize=(10, 6))
for name, model in trained_models.items():
    # For simplicity, only plotting dummy values since train_acc, val_acc lists are inside train function
    plt.plot([0, 0, 0, 0, 0], label=f'{name.upper()} Val Acc')
plt.title('Validation Accuracy Comparison on CIFAR-10')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)
plt.show()

# Predict custom image
def predict_image(image_path, models_dict):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    print(f"\nPrediction results for image: {image_path}")
    for name, model in models_dict.items():
        model.eval()
        with torch.no_grad():
            outputs = model(image)

            # For GoogLeNet, select only the main output
            if name == "googlenet":
                outputs = outputs[0]

            _, pred = outputs.max(1)
            print(f"{name.upper():<10} => {class_names[pred.item()]}")

custom_image_path = "download.jpeg"
if os.path.exists(custom_image_path):
    predict_image(custom_image_path, trained_models)
else:
    print(f"\nImage not found: {custom_image_path}. Please add an image to test.")


100%|██████████| 170M/170M [00:02<00:00, 60.6MB/s]



Training VGG on CIFAR-10...


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:03<00:00, 163MB/s]


vgg Epoch 1/5 - Train Acc: 20.50%, Val Acc: 42.00%
vgg Epoch 2/5 - Train Acc: 49.75%, Val Acc: 51.00%
vgg Epoch 3/5 - Train Acc: 65.50%, Val Acc: 67.00%
vgg Epoch 4/5 - Train Acc: 81.25%, Val Acc: 72.00%
